# Init

In [1]:
from pandas_data_reader_service_core.modules.finam.search_service import Search, Market
from pandas_data_reader_service_core.modules.finam.stock_info import FinamStockInfo
from pandas_data_reader_service_core.service import PandasDataReaderService as pdrs, StockInfo, TimeFrame

from datetime import date

import plotly.graph_objects as go

import pandas as pd
from datetime import datetime

In [34]:
stock = "BTCUSD"
market = Market.CRYPTO_CURRENCIES

date_from = date(2015,1,1)
date_till = date(2020,2,1)
tf = TimeFrame.WEEKLY

# Load data

In [35]:
stock_list = Search.by_code(stock, market)
stock_list

,name,code,market
id,,,
499054,Bitcoin / U.S. dollar,BTCUSD,520
484429,BTC-USD,BTCUSD,520


In [36]:
stock_info = StockInfo.init_for_Finam(FinamStockInfo(stock_list.index[0]))
print(stock_info)

Source StockSource.Finam source StockInfo Market 520 code BTCUSD index 499054


In [37]:
data_df = pdrs().get(stock=stock_info,  date_from=date_from,date_to=date_till,time_frame=tf,short_col_name=True)
data_df

,O,H,L,C,V
DT,,,,,
2015-06-29,263.83,275.25,252.77,271.74,31698
2015-07-06,272.17,316.52,264.41,312.16,68869
2015-07-13,312.19,312.91,272.64,275.61,50837
2015-07-20,275.60,294.49,273.34,293.89,35331
2015-07-27,293.88,298.00,277.26,282.62,35304
...,...,...,...,...,...
2019-12-30,7372.79,7495.00,6853.53,7356.70,22793
2020-01-06,7352.12,8463.57,7342.46,8180.76,45556
2020-01-13,8180.75,9188.10,8039.00,8696.60,51236


In [38]:
fig = go.Figure(data=[go.Candlestick(x=data_df.index,
                open=data_df['O'],
                high=data_df['H'],
                low=data_df['L'],
                close=data_df['C'])])
fig.update_yaxes(fixedrange=False)
fig.update_layout(height=1000)
fig.show()

# Find Rules

## Support functions

In [138]:
from __future__ import annotations

class candle_chart:
    def __init__(self,df: pd.DataFrame) -> None:
        self.fig = go.Figure(data=[go.Candlestick(x=df.index,
            open= df['O'],
            high= df['H'],
            low=  df['L'],
            close=df['C'])])
        self.fig.update_yaxes(fixedrange=False)
        self.fig.update_layout(height=1000)
        self.max = df.H.max()
        pass

    def add_marker(self, marker_candle:pd.Series)->candle_chart:
        self.fig.add_shape(type="line",
            x0=marker_candle.name, x1=marker_candle.name, y0=0, y1=marker_candle["L"]*0.9,
            line=dict(
                color="MediumPurple",
                width=4,
                dash="dot",
            )
        )

      
        self.fig.add_shape(type="line",
            x0=marker_candle.name, x1=marker_candle.name, y0=marker_candle["H"]*1.1, y1=self.max,
            line=dict(
                color="MediumPurple",
                width=4,
                dash="dot",
            )
        )
        return self
    def add_wave(self, start_candle:pd.Series, end_candle:pd.Series, main_wave: bool = True)->candle_chart:
        if end_candle.C > start_candle.C:
            color = "Green"
        else:
            color = "Red"
        if main_wave:
            dash = "solid"
            y=start_candle.L, end_candle.H
        else:
            dash = "dashdot"
            y=start_candle.H, end_candle.L
        self.fig.add_shape(type="line",
            x0=start_candle.name, x1=end_candle.name, y0=y[0], y1=y[1],
            line=dict(
                color=color,
                width=2,
                dash=dash,
            )
        )
        return self
    def show(self):
        self.fig.show()

In [139]:
zoom = (pd.Timestamp(year=2018, month=2,day=5),  pd.Timestamp(year=2019,month=12,day=9))
zoom_df = data_df.loc[[dt for dt in data_df.index if (dt >= zoom[0]) & (dt <= zoom[1])]]

## Wave1

### Define wave

#### Define Start Wave 1 SW1

In [140]:
SW1_dt = pd.Timestamp(year=2019,month=1,day=28)
SW1_dt

Timestamp('2019-01-28 00:00:00')

In [141]:
SW1_candle = data_df.loc[SW1_dt]
SW1_candle

O     3530.00
H     3536.77
L     3322.19
C     3414.48
V    38721.00
Name: 2019-01-28 00:00:00, dtype: float64

In [142]:
candle_chart(zoom_df).add_marker(SW1_candle).show()

#### Define End Wave 1 EW1

In [143]:
EW1_dt = pd.Timestamp(year=2019,month=4,day=1)
EW1_dt

Timestamp('2019-04-01 00:00:00')

In [144]:
EW1_candle = data_df.loc[EW1_dt]
EW1_candle

O      4093.74
H      5344.60
L      4052.56
C      5193.70
V    106758.00
Name: 2019-04-01 00:00:00, dtype: float64

In [145]:
candle_chart(zoom_df).add_marker(SW1_candle).add_marker(EW1_candle).show()

#### Define range wave 1 RW1

In [146]:
RW1 = zoom_df.loc[[dt for dt in zoom_df.index if (dt >= SW1_dt) & (dt <= EW1_dt)]]
RW1

,O,H,L,C,V
DT,,,,,
2019-01-28,3530.00,3536.77,3322.19,3414.48,38721
2019-02-04,3417.35,3710.25,3330.00,3644.35,36882
2019-02-11,3647.50,3663.34,3531.01,3622.37,43729
2019-02-18,3625.60,4190.00,3617.18,3730.68,69344
2019-02-25,3732.60,3897.62,3658.51,3788.00,37020
2019-03-04,3789.70,3950.25,3670.00,3898.74,41787
2019-03-11,3900.58,4040.00,3775.01,3963.90,34663
2019-03-18,3965.50,4055.35,3919.63,3969.34,28632
2019-03-25,3969.00,4130.00,3850.45,4095.65,34194


In [147]:
candle_chart(zoom_df).add_marker(SW1_candle).add_marker(EW1_candle).add_wave(SW1_candle,EW1_candle).show()

### Rules

#### End > Start

In [148]:
EW1_candle.C > SW1_candle.O 

True

#### EW1 is max on range W1

In [149]:
(RW1[["H","L"]] > EW1_candle.H).any(axis=1)

DT
2019-01-28    False
2019-02-04    False
2019-02-11    False
2019-02-18    False
2019-02-25    False
2019-03-04    False
2019-03-11    False
2019-03-18    False
2019-03-25    False
2019-04-01    False
dtype: bool

## Wave 2

### Define wave

#### Define Start Wave 2 SW2

In [150]:
SW2_dt = pd.Timestamp(year=2019,month=4,day=1)
SW2_candle = zoom_df.loc[SW2_dt]
print(SW2_candle.equals(EW1_candle))
SW2_candle

True


O      4093.74
H      5344.60
L      4052.56
C      5193.70
V    106758.00
Name: 2019-04-01 00:00:00, dtype: float64

In [156]:
candle_chart(zoom_df).add_marker(SW1_candle).add_marker(EW1_candle).add_wave(SW1_candle,EW1_candle).add_marker(SW2_candle).show()

#### Define End Wave 2 SW2

In [152]:
EW2_dt = pd.Timestamp(year=2019,month=4,day=22)
EW2_candle = zoom_df.loc[EW2_dt]
EW2_candle

O     5295.96
H     5623.30
L     4991.42
C     5148.38
V    56146.00
Name: 2019-04-22 00:00:00, dtype: float64

In [153]:
candle_chart(zoom_df).add_marker(SW1_candle).add_marker(EW1_candle).add_wave(SW1_candle,SW2_candle).add_marker(SW2_candle).add_marker(EW2_candle).show()

#### Define range wave 1 RW1

In [154]:
RW2 = zoom_df.loc[[dt for dt in zoom_df.index if (dt >= SW2_dt) & (dt <= EW2_dt)]]
RW2

,O,H,L,C,V
DT,,,,,
2019-04-01,4093.74,5344.60,4052.56,5193.70,106758
2019-04-08,5182.92,5466.06,4912.00,5162.00,58210
2019-04-15,5163.00,5361.60,4950.00,5295.98,37342
2019-04-22,5295.96,5623.30,4991.42,5148.38,56146


In [155]:
candle_chart(zoom_df).add_marker(SW1_candle).add_marker(EW1_candle).add_wave(SW1_candle,SW2_candle).add_marker(SW2_candle).add_marker(EW2_candle).add_wave(SW2_candle,EW2_candle,main_wave=False).show()

### Rules

#### End wave 2 is less than start start wave 2

In [ ]:
EW2_candle.L < SW2_candle.H